# 🧠 ТАРС v3 — Полное обучение

| Параметр | Значение |
|----------|----------|
| Модель | 2048d, 24 слоя (~1B params) |
| Квантизация | 1.58-bit BitNet |
| Голос | Whisper (RU) + Piper (RU) |
| Время | 8-15ч (A100) / 15-24ч (T4) |

**⚠️ Рекомендуется A100.** Runtime → Change runtime type → A100

**⚠️ Сначала запустите `TARS_Test.ipynb`** чтобы убедиться что всё работает!

## 1. Проверка GPU

In [ ]:
import torch
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f'🎮 {gpu} — {vram:.0f} GB VRAM')
    if vram >= 40:
        print('⚡ A100 — отлично! ~8-10 часов')
    elif vram >= 15:
        print('✅ T4/L4 — хорошо. ~15-24 часа')
    else:
        print('⚠️ Мало VRAM — возможны OOM ошибки')
else:
    print('❌ GPU не найден! Runtime → Change runtime type → GPU')

## 2. Загрузка проекта

In [ ]:
# ═══ Подключение Google Drive (для сохранения моделей) ═══
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ═══ Загрузка проекта ═══
# Раскомментируйте ОДИН из вариантов:

# Вариант A: С Google Drive
!cp -r /content/drive/MyDrive/TarsSSM-Py /content/TarsSSM-Py

# Вариант B: Git clone
# !git clone https://github.com/<username>/TarsSSM-Py.git /content/TarsSSM-Py

# Вариант C: ZIP upload
# from google.colab import files
# uploaded = files.upload()
# !unzip -qo TarsSSM-Py.zip -d /content/

In [ ]:
%cd /content/TarsSSM-Py
!ls mega_train.py launch_tars.py brain/ training/

## 3. 🚀 Полное обучение

Выберите режим ниже.

**Модели автоматически сохраняются** в `models/mamba2/` и `models/tars_v3/` по каждой эпохе.

In [ ]:
#@title Параметры обучения { run: "auto" }
skip_voice = False  #@param {type:"boolean"}
skip_download = False  #@param {type:"boolean"}
resume = False  #@param {type:"boolean"}

cmd = 'python mega_train.py'
if skip_voice:
    cmd += ' --skip-voice'
if skip_download:
    cmd += ' --skip-download'
print(f'Команда: {cmd}')
if skip_voice:
    print('⏱ Время: ~5-8 часов (без голоса)')
else:
    print('⏱ Время: ~8-15 часов (полный пайплайн)')

In [ ]:
# ═══════════════════════════════════════
#  ЗАПУСК ОБУЧЕНИЯ
# ═══════════════════════════════════════

cmd_parts = ['python', 'mega_train.py']
if skip_voice:
    cmd_parts.append('--skip-voice')
if skip_download:
    cmd_parts.append('--skip-download')

import subprocess
result = subprocess.run(cmd_parts)
print(f'\nКод возврата: {result.returncode}')

## 4. 💾 Сохранение на Google Drive

In [ ]:
import shutil
from pathlib import Path

src = Path('/content/TarsSSM-Py/models/tars_v3')
dst = Path('/content/drive/MyDrive/TarsModels')
dst.mkdir(parents=True, exist_ok=True)

if src.exists():
    for f in src.glob('*.pt'):
        size = f.stat().st_size / 1024 / 1024
        print(f'  📦 {f.name} ({size:.1f} MB) → Drive...')
        shutil.copy2(str(f), str(dst / f.name))
    
    # Также сохраняем промежуточные чекпоинты
    mamba_dir = Path('/content/TarsSSM-Py/models/mamba2')
    if mamba_dir.exists():
        mamba_dst = dst / 'mamba2'
        mamba_dst.mkdir(exist_ok=True)
        for f in mamba_dir.glob('*.pt'):
            shutil.copy2(str(f), str(mamba_dst / f.name))
    
    print(f'\n✅ Модели сохранены: {dst}')
    print('   Теперь можно закрывать Colab — модели на Drive!')
else:
    print('⚠️ models/tars_v3/ не найдена')

## 5. Проверка

In [ ]:
!python launch_tars.py --check

In [ ]:
# Размеры моделей
!ls -lh models/tars_v3/ 2>/dev/null || echo 'Модели ещё не обучены'
!echo '---'
!ls -lh models/mamba2/*.pt 2>/dev/null || echo 'Нет чекпоинтов mamba2'

In [ ]:
# Последние 30 строк лога
!tail -30 mega_train.log

---

## 🔄 Продолжение после разрыва сессии

Если Colab отключился:
1. Подключите GPU заново
2. Запустите ячейки 1-2 (Drive + проект)
3. Восстановите чекпоинты с Drive:

In [ ]:
# ═══ Восстановление после разрыва ═══
import shutil
from pathlib import Path

drive_models = Path('/content/drive/MyDrive/TarsModels')
local_models = Path('/content/TarsSSM-Py/models')

if drive_models.exists():
    # Восстановить mamba2 чекпоинты
    mamba_src = drive_models / 'mamba2'
    if mamba_src.exists():
        mamba_dst = local_models / 'mamba2'
        mamba_dst.mkdir(parents=True, exist_ok=True)
        for f in mamba_src.glob('*.pt'):
            shutil.copy2(str(f), str(mamba_dst / f.name))
            print(f'  ✅ {f.name}')
    
    # Восстановить tars_v3
    for f in drive_models.glob('*.pt'):
        dst = local_models / 'tars_v3'
        dst.mkdir(parents=True, exist_ok=True)
        shutil.copy2(str(f), str(dst / f.name))
        print(f'  ✅ {f.name}')
    
    print('\n✅ Чекпоинты восстановлены. Продолжаем:')
    print('   !python mega_train.py --skip-download')
else:
    print('⚠️ Нет сохранённых моделей на Drive')